In [1]:
import games
from notebook import psource, pseudocode

In [479]:
class Mancala(games.Game):
    """A game is similar to a problem, but it has a utility for each
    state and a terminal test instead of a path cost and a goal
    test. To create a game, subclass this class and implement actions,
    result, utility, and terminal_test. You may override display and
    successors or you can inherit their default methods. You will also
    need to set the .initial attribute to the initial state; this can
    be done in the constructor."""

    def __init__(self, board = [4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 0], player_turn=True):
        #4 marbels per cup, on each player's side
        self.board = board
        #True for player 1 and False for player 2
        self.player_turn = player_turn
        
    
    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        moves = []
        cups = state.board
        if state.player_turn==True:
            board_range = range(0,6)
        else:
            board_range = range(7,13)
        for m in board_range:
            if cups[m] != 0 and m%7!=6:
                moves += [m]
        return moves

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        #getting the marbles in the cup
        marblesInCup = state.board[move]
        currentPlayer = state.player_turn
        
        if currentPlayer:
            scoreSpot = 6
        else:
            scoreSpot = 13

        state.board[move]=0
        initialPlace = move
        for i in range(0,marblesInCup):
            boardIndex = move%14
            #Makes sure that the players don't score on the opponant score board
            if currentPlayer==True and boardIndex==12:
                move+=1
            elif currentPlayer==False and boardIndex==5:
                move+=1
            #initial cup will always be empty & skipped over
            elif initialPlace == boardIndex+1:
                move+=1
            move+=1
            state.board[move%14]+=1

        lastSpot = move%14
        #If the last piece you drop is in an empty hole on your side,
        #you capture that piece and any pieces in the hole directly opposite.
        if state.board[lastSpot] == 1:
            if (currentPlayer and lastSpot < 6) or (not currentPlayer and lastSpot>6 and lastSpot<13):
                state.board[scoreSpot] = state.board[lastSpot] + state.board[(lastSpot+7)%14]
                state.board[lastSpot]=0
                state.board[(lastSpot+7)%14]=0

        #the player changes only when the current player's marble don't land on their score cup
        if (currentPlayer and lastSpot!=6) or (not currentPlayer and lastSpot!=13):
            state.player_turn = not currentPlayer
    
        return state

    def utility(self, state, player):
        """Return the value of this final state to player."""
        sum = 0
        if (state.board[0] == 0 and state.board[1] == 0 and state.board[2] == 0 and state.board[3] == 0 and state.board[4] == 0 and state.board[5] == 0) :
            for i in range(7,13) :
                sum += state.board[i]
            state.board[13] += sum

        elif (state.board[7] == 0 and state.board[8] == 0 and state.board[9] == 0 and state.board[10] == 0 and state.board[11] == 0 and state.board[12] == 0) :
            for i in range(6) :
                sum += state.board[i]
            state.board[6] += sum
            
        return self.check_winner(state.board)
        
    def check_winner(self, board) :
        """Checks which player won. Player 1 wins if it returns 1, Player 2 wins if it returns 2."""
        if board[6] > board[13] :
            return 1
        elif board[6] < board[13] :
            return -1
        elif board[6] == board[13] :
            return 0

    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        if ((state.board[0] == 0 and state.board[1] == 0 and state.board[2] == 0 and state.board[3] == 0 and state.board[4] == 0 and state.board[5] == 0) or (state[7].board == 0 and state[8].board == 0 and state.board[9] == 0 and state.board[10] == 0 and state.board[11] == 0 and state.board[12] == 0)) :
            return True
        else :
            return False

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move

    def display(self, state):
        """Print or otherwise display the state."""
        print(state)

    def __repr__(self):
        board = "Player 2"
        board += "\n\t6\t5\t4\t3\t2\t1"
        board += "\n------------------------------------------------------------\n"
        board += str(self.board[13]) + "\t"
        for elem in range(12,6,-1):
            board += str(self.board[elem]) + "\t"
        board += "\n\t"
        for elem in range(0,6):
            board += str(self.board[elem]) + "\t"
        board += str(self.board[6])
        board += "\n------------------------------------------------------------"
        board += "\n\t1\t2\t3\t4\t5\t6\n"
        board += "Player 1\n"        
        return board

    def play_game(self, *players):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                if self.terminal_test(state):
                    self.display(state)
                    return self.utility(state, self.to_move(self.initial))

In [480]:
#If the last piece you drop is in your own store, you get a free turn.
mancala = Mancala()
print(mancala.board)
print("Current player turn:",mancala.player_turn)
print(mancala.result(mancala,2))
print(mancala.board)
print("Next player turn:", mancala.player_turn)

[4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 0]
Current player turn: True
Player 2
	6	5	4	3	2	1
------------------------------------------------------------
0	4	4	4	4	4	4	
	4	4	0	5	5	5	1
------------------------------------------------------------
	1	2	3	4	5	6
Player 1

[4, 4, 0, 5, 5, 5, 1, 4, 4, 4, 4, 4, 4, 0]
Next player turn: True


In [481]:
#Check to see if the player doesn't score on the opponant score board
test1 = Mancala([4, 4, 4, 4, 4, 12, 0, 4, 4, 4, 4, 4, 4, 0])
print("Current player turn:",test1.player_turn)
print(mancala_1.result(test1,5))

Current player turn: True
Player 2
	6	5	4	3	2	1
------------------------------------------------------------
0	5	5	5	5	5	5	
	5	5	5	5	5	0	1
------------------------------------------------------------
	1	2	3	4	5	6
Player 1



In [482]:
#If the last piece you drop is in an empty hole on your side,
#you capture that piece and any pieces in the hole directly opposite.
mancala_2 = Mancala([4, 4, 4, 4, 4, 4, 0, 4, 4, 0, 12, 4, 4, 0],False)
print("Current player turn:",mancala_2.player_turn)
print("*****Initial Board where player 2 picks up from cup 4:*****")
print(mancala_2.__repr__)
print(mancala_2.result(mancala_1,10))
print(mancala_2.board)
print("Next player turn:", mancala_2.player_turn)

Current player turn: False
*****Initial Board where player 2 picks up from cup 4:*****
<bound method Mancala.__repr__ of Player 2
	6	5	4	3	2	1
------------------------------------------------------------
0	4	4	12	0	4	4	
	4	4	4	4	4	4	0
------------------------------------------------------------
	1	2	3	4	5	6
Player 1
>
Player 2
	6	5	4	3	2	1
------------------------------------------------------------
1	5	5	0	5	5	5	
	5	5	0	5	5	0	2
------------------------------------------------------------
	1	2	3	4	5	6
Player 1

[4, 4, 4, 4, 4, 4, 0, 4, 4, 0, 12, 4, 4, 0]
Next player turn: False


* `actions(self, state)`: Given a game state, this method generates all the legal actions possible from this state, as a list or a generator. Returning a generator rather than a list has the advantage that it saves space and you can still operate on it as a list.


* `result(self, state, move)`: Given a game state and a move, this method returns the game state that you get by making that move on this game state.


* `utility(self, state, player)`: Given a terminal game state and a player, this method returns the utility for that player in the given terminal game state. While implementing this method assume that the game state is a terminal game state. The logic in this module is such that this method will be called only on terminal game states.


* `terminal_test(self, state)`: Given a game state, this method should return `True` if this game state is a terminal state, and `False` otherwise.


* `to_move(self, state)`: Given a game state, this method returns the player who is to play next. This information is typically stored in the game state, so all this method does is extract this information and return it.


* `display(self, state)`: This method prints/displays the current state of the game.